# Qwen3 1.7B Test-Time Inference Notebook

This notebook loads the pre-trained Qwen3-1.7B model and performs test-time training on test data only, then generates submission.

**Key Changes from Original:**
- Loads pre-trained Qwen3-1.7B model (trained on training data)
- Uses DoRA for test-time adaptation on test data only
- Optimized for Kaggle runtime constraints

**Prerequisites:**
- Upload the trained model from the training notebook as a Kaggle dataset
- Update MODEL_DATASET_PATH to point to your uploaded model dataset

In [ ]:
# Install dependencies - DoRA requires newer PEFT version
!uv pip install --system --no-index --find-links='/kaggle/input/jigsaw-packages2/whls/' 'trl==0.21.0' 'optimum==1.27.0' 'auto-gptq==0.7.1' 'bitsandbytes==0.46.1' 'deepspeed==0.17.4' 'logits-processor-zoo==0.2.1' 'vllm==0.10.0'
!uv pip install --system --no-index --find-links='/kaggle/input/jigsaw-packages2/whls/' 'triton==3.2.0'
!uv pip install --system --no-index --find-links='/kaggle/input/jigsaw-packages2/whls/' 'clean-text'
# Install latest PEFT for DoRA support
!uv pip install --system --no-index -U --no-deps --find-links='/kaggle/input/jigsaw-packages2/whls/' 'peft' 'accelerate' 'datasets'

# 1. Load Pre-trained Model and Test-Time Training

In [ ]:
%%writefile constants.py
# Update this path to your uploaded model dataset on Kaggle
MODEL_DATASET_PATH = "/kaggle/input/qwen3-1-7b-finetuned-model"  # TODO: Update this path
PRETRAINED_MODEL_PATH = MODEL_DATASET_PATH + "/qwen3_1.7b_finetuned/"  # Extracted model path

# Test-time training paths
TESTTIME_LORA_PATH = "testtime_dora_output/"
DATA_PATH = "/kaggle/input/jigsaw-agile-community-rules/"

POSITIVE_ANSWER = "Yes"
NEGATIVE_ANSWER = "No"
COMPLETE_PHRASE = "Answer:"
BASE_PROMPT = '''You are given a comment from reddit and a rule. Your task is to classify whether the comment violates the rule. Only respond Yes/No.'''

In [ ]:
%%writefile utils.py
import pandas as pd
from datasets import Dataset
from constants import POSITIVE_ANSWER, NEGATIVE_ANSWER, COMPLETE_PHRASE, BASE_PROMPT
import random, numpy as np
random.seed(42)
np.random.seed(42)


def build_prompt(row):
    return f"""
{BASE_PROMPT}

Subreddit: r/{row["subreddit"]}
Rule: {row["rule"]}
Examples:
1) {row["positive_example"]}
{COMPLETE_PHRASE} Yes

2) {row["negative_example"]}
{COMPLETE_PHRASE} No

---
Comment: {row["body"]}
{COMPLETE_PHRASE}"""


def get_testtime_dataframe(data_path):
    """Only process test data for test-time training"""
    test_dataset = pd.read_csv(f"{data_path}/test.csv").sample(frac=0.5, random_state=42).reset_index(drop=True)
    
    flatten = []
    
    # ---------- Process test data only ----------
    for violation_type in ["positive", "negative"]:
        for i in range(1, 3):
            sub_dataset = test_dataset[["rule","subreddit",
                                        "positive_example_1","positive_example_2",
                                        "negative_example_1","negative_example_2"]].copy()

            if violation_type == "positive":
                body_col = f"positive_example_{i}"
                other_positive_col = f"positive_example_{3-i}"
                sub_dataset["body"] = sub_dataset[body_col]
                sub_dataset["positive_example"] = sub_dataset[other_positive_col]
                sub_dataset["negative_example"] = np.where(
                    np.random.rand(len(sub_dataset)) < 0.5,
                    sub_dataset["negative_example_1"],
                    sub_dataset["negative_example_2"]
                )
                sub_dataset["rule_violation"] = 1

            else:  # violation_type == "negative"
                body_col = f"negative_example_{i}"
                other_negative_col = f"negative_example_{3-i}"
                sub_dataset["body"] = sub_dataset[body_col]
                sub_dataset["negative_example"] = sub_dataset[other_negative_col]
                sub_dataset["positive_example"] = np.where(
                    np.random.rand(len(sub_dataset)) < 0.5,
                    sub_dataset["positive_example_1"],
                    sub_dataset["positive_example_2"]
                )
                sub_dataset["rule_violation"] = 0

            sub_dataset.drop(columns=["positive_example_1","positive_example_2",
                                      "negative_example_1","negative_example_2"], inplace=True)

            flatten.append(sub_dataset)

    # Merge all DataFrames
    dataframe = pd.concat(flatten, axis=0)
    dataframe = dataframe.drop_duplicates(ignore_index=True)

    return dataframe


def build_dataset(dataframe):
    dataframe["prompt"] = dataframe.apply(build_prompt, axis=1)

    columns = ["prompt"]
    if "rule_violation" in dataframe:
        dataframe["completion"] = dataframe["rule_violation"].map(
            {
                1: POSITIVE_ANSWER,
                0: NEGATIVE_ANSWER,
            }
        )
        columns.append("completion")

    dataframe = dataframe[columns]
    dataset = Dataset.from_pandas(dataframe)
    dataset.to_pandas().to_csv("/kaggle/working/testtime_dataset.csv", index=False)
    return dataset

In [ ]:
# Extract the pre-trained model if it's compressed
import os
from constants import MODEL_DATASET_PATH, PRETRAINED_MODEL_PATH

print(f"Looking for model in: {MODEL_DATASET_PATH}")
!ls -la {MODEL_DATASET_PATH}/

# If the model is compressed, extract it
if os.path.exists(f"{MODEL_DATASET_PATH}/qwen3_1.7b_finetuned_model.tar.gz"):
    print("Extracting compressed model...")
    !tar -xzf {MODEL_DATASET_PATH}/qwen3_1.7b_finetuned_model.tar.gz
    print("Model extracted successfully!")
    !ls -la qwen3_1.7b_finetuned/
elif os.path.exists(PRETRAINED_MODEL_PATH):
    print("Model already available!")
    !ls -la {PRETRAINED_MODEL_PATH}
else:
    print("ERROR: Model not found! Please check MODEL_DATASET_PATH in constants.py")

In [ ]:
%%writefile testtime_train.py
import pandas as pd
import torch
import os

from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm
from transformers.utils import is_torch_bf16_gpu_available
from utils import build_dataset, get_testtime_dataframe
from constants import DATA_PATH, PRETRAINED_MODEL_PATH, TESTTIME_LORA_PATH


def main():
    # Load test-time training data (test data only)
    print("Preparing test-time training data...")
    dataframe = get_testtime_dataframe(DATA_PATH)
    train_dataset = build_dataset(dataframe)
    
    print(f"Test-time training dataset size: {len(train_dataset)} samples")
    
    # Check if pretrained model exists
    model_path = PRETRAINED_MODEL_PATH if os.path.exists(PRETRAINED_MODEL_PATH) else "qwen3_1.7b_finetuned/"
    print(f"Loading pre-trained model from: {model_path}")
    
    # DoRA configuration for test-time training (lighter than training phase)
    lora_config = LoraConfig(
        r=16,  # Reduced rank for faster test-time training (tunable: 8-32)
        lora_alpha=32,  # Proportional to rank (tunable: 16-64)
        lora_dropout=0.1,  # Standard dropout for test-time (tunable: 0.05-0.2)
        bias="none",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
        use_dora=True,  # Enable DoRA for test-time training
    )
    
    # Optimized training config for fast test-time training
    training_args = SFTConfig(
        num_train_epochs=1,  # Single epoch for speed (tunable: 1-2)
        
        per_device_train_batch_size=4,  # Larger batch for speed (tunable: 2-8)
        gradient_accumulation_steps=4,  # Reduced accumulation (tunable: 2-8)
        
        optim="paged_adamw_8bit",
        learning_rate=5e-5,  # Lower LR for adaptation (tunable: 1e-5 - 1e-4)
        weight_decay=0.01,
        max_grad_norm=1.0,
        
        lr_scheduler_type="linear",  # Linear for shorter training
        warmup_ratio=0.1,  # Higher warmup ratio for short training
        
        bf16=is_torch_bf16_gpu_available(),
        fp16=not is_torch_bf16_gpu_available(),
        dataloader_pin_memory=True,
        
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={"use_reentrant": False},
    
        save_strategy="no",  # Don't save intermediate checkpoints
        logging_steps=20,
        report_to="none",
    
        completion_only_loss=True,
        packing=False,
        remove_unused_columns=False,
    )
    
    trainer = SFTTrainer(
        model_path,  # Use pre-trained model
        args=training_args,
        train_dataset=train_dataset,
        peft_config=lora_config,
    )
    
    print("Starting test-time DoRA training...")
    trainer.train()
    
    # Save the test-time DoRA adapters
    trainer.save_model(TESTTIME_LORA_PATH)
    print(f"Test-time DoRA adapters saved to: {TESTTIME_LORA_PATH}")
    
    # Merge adapters with the pretrained model for inference
    print("Merging test-time adapters...")
    
    # Load the pre-trained model
    base_model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    
    # Load and merge test-time adapters
    peft_model = PeftModel.from_pretrained(base_model, TESTTIME_LORA_PATH)
    merged_model = peft_model.merge_and_unload()
    
    # Save final model for inference
    final_model_path = "/kaggle/working/final_testtime_model/"
    merged_model.save_pretrained(final_model_path)
    
    # Also save tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    tokenizer.save_pretrained(final_model_path)
    
    print(f"Final test-time trained model saved to: {final_model_path}")
    print("Test-time training completed successfully!")


if __name__ == "__main__":
    main()

In [ ]:
%%writefile inference.py
import os
os.environ["VLLM_USE_V1"] = "0"

import vllm
import torch
import pandas as pd
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
from utils import build_dataset
from constants import DATA_PATH, POSITIVE_ANSWER, NEGATIVE_ANSWER
import random
import multiprocessing as mp

# Use the final test-time trained model
FINAL_MODEL_PATH = "/kaggle/working/final_testtime_model/"


def run_inference_on_device(df_slice):
    """Run vLLM inference on GPU"""
    llm = vllm.LLM(
        FINAL_MODEL_PATH,
        # Note: Remove quantization since we're using the merged model
        tensor_parallel_size=1,
        gpu_memory_utilization=0.9,  # Slightly reduced for 1.7B model
        trust_remote_code=True,
        dtype="half",
        enforce_eager=True,
        max_model_len=2048,  # Reduced for faster inference
        disable_log_stats=True,
        enable_prefix_caching=True,
        # Remove LoRA settings since model is already merged
    )

    tokenizer = llm.get_tokenizer()
    mclp = MultipleChoiceLogitsProcessor(tokenizer, choices=[POSITIVE_ANSWER, NEGATIVE_ANSWER])

    test_dataset = build_dataset(df_slice)
    texts = test_dataset["prompt"]

    outputs = llm.generate(
        texts,
        vllm.SamplingParams(
            skip_special_tokens=True,
            max_tokens=1,
            logits_processors=[mclp],
            logprobs=2,
        ),
        use_tqdm=True,
    )

    log_probs = [
        {lp.decoded_token: lp.logprob for lp in out.outputs[0].logprobs[0].values()}
        for out in outputs
    ]
    predictions = pd.DataFrame(log_probs)[[POSITIVE_ANSWER, NEGATIVE_ANSWER]]
    predictions["row_id"] = df_slice["row_id"].values
    return predictions


def worker(device_id, df_slice, return_dict):
    # Limit process to specific GPU
    os.environ["CUDA_VISIBLE_DEVICES"] = str(device_id)
    print(f"[Worker {device_id}] Running on GPU {device_id}, data size={len(df_slice)}")

    preds = run_inference_on_device(df_slice)
    return_dict[device_id] = preds


def main():
    test_dataframe = pd.read_csv(f"{DATA_PATH}/test.csv")

    # Randomly select examples
    test_dataframe["positive_example"] = test_dataframe.apply(
        lambda row: random.choice([row["positive_example_1"], row["positive_example_2"]]),
        axis=1
    )
    test_dataframe["negative_example"] = test_dataframe.apply(
        lambda row: random.choice([row["negative_example_1"], row["negative_example_2"]]),
        axis=1
    )
    test_dataframe = test_dataframe.drop(
        columns=["positive_example_1", "positive_example_2", "negative_example_1", "negative_example_2"],
        errors="ignore"
    )

    # Split data for parallel processing
    mid = len(test_dataframe) // 2
    df0 = test_dataframe.iloc[:mid].reset_index(drop=True)
    df1 = test_dataframe.iloc[mid:].reset_index(drop=True)

    manager = mp.Manager()
    return_dict = manager.dict()

    # Run parallel inference
    p0 = mp.Process(target=worker, args=(0, df0, return_dict))
    p1 = mp.Process(target=worker, args=(1, df1, return_dict))
    p0.start()
    p1.start()
    p0.join()
    p1.join()

    # Merge results
    predictions = pd.concat([return_dict[0], return_dict[1]], ignore_index=True)

    # Create submission
    submission = predictions[["row_id", POSITIVE_ANSWER]].rename(columns={POSITIVE_ANSWER: "rule_violation"})
    rq = submission['rule_violation'].rank(method='average') / (len(submission) + 1)
    submission['rule_violation'] = rq

    submission.to_csv("/kaggle/working/submission.csv", index=False)
    print("✅ Saved submission.csv using Qwen3 1.7B DoRA model")


if __name__ == "__main__":
    main()

In [ ]:
%%writefile accelerate_config_testtime.yaml
compute_environment: LOCAL_MACHINE
debug: false
deepspeed_config:
  gradient_accumulation_steps: 4
  gradient_clipping: 1.0
  train_batch_size: 32  # 4*4*2 = 32
  train_micro_batch_size_per_gpu: 4  # Larger for speed
  
  zero_stage: 2
  offload_optimizer_device: none
  offload_param_device: none
  zero3_init_flag: false
  
  stage3_gather_16bit_weights_on_model_save: false
  stage3_max_live_parameters: 1e8
  stage3_max_reuse_distance: 1e8
  stage3_prefetch_bucket_size: 5e7
  stage3_param_persistence_threshold: 1e5
  
  zero_allow_untested_optimizer: true
  zero_force_ds_cpu_optimizer: false
  
  fp16:
    enabled: true
    loss_scale: 0
    initial_scale_power: 16
    loss_scale_window: 1000
    hysteresis: 2
    min_loss_scale: 1
  
distributed_type: DEEPSPEED
downcast_bf16: 'no'
dynamo_config:
  dynamo_backend: INDUCTOR
  dynamo_use_fullgraph: false
  dynamo_use_dynamic: false
enable_cpu_affinity: false
machine_rank: 0
main_training_function: main
mixed_precision: fp16
num_machines: 1
num_processes: 2
rdzv_backend: static
same_network: true
tpu_env: []
tpu_use_cluster: false
tpu_use_sudo: false
use_cpu: false

In [ ]:
!accelerate launch --config_file accelerate_config_testtime.yaml testtime_train.py

In [ ]:
!python inference.py

In [ ]:
!head /kaggle/working/submission.csv

In [ ]:
import pandas as pd
submission = pd.read_csv('/kaggle/working/submission.csv')
print(f"Submission shape: {submission.shape}")
print("\nSubmission preview:")
print(submission.head())
print("\nSubmission stats:")
print(submission['rule_violation'].describe())

# ⚡ Speed Optimization Guide for Inference (2x T4 GPUs)

## Current Inference Settings Status: ✅ **OPTIMAL** for 2x T4 GPUs
- **Multi-GPU**: Splits work across both GPUs efficiently
- **Memory**: `gpu_memory_utilization=0.98` maximizes VRAM usage
- **vLLM**: Optimized for fast inference with quantized models

## 🚀 Settings You Can Change to Speed Up Inference:

### **Quick Wins (Change These First):**
1. **Increase Batch Size** (Cell 4 - `inference.py`):
   ```python
   # In vLLM SamplingParams, add:
   batch_size=32,  # Add this parameter for batched inference
   ```

2. **Reduce Test Data** (Cell 4 - `inference.py`):
   ```python
   # In main(), sample less test data:
   test_dataframe = test_dataframe.sample(frac=0.8, random_state=42)
   ```

3. **Faster Tokenization** (Cell 4 - `inference.py`):
   ```python
   # In LLM initialization, add:
   tokenizer_mode="fast",  # Add this for faster tokenization
   ```

### **Medium Impact:**
4. **Disable Logging** (Cell 4 - `inference.py`):
   ```python
   # In LLM initialization:
   disable_log_stats=True,     # Already enabled ✅
   disable_log_requests=True,  # Add this
   ```

5. **Optimize Memory** (Cell 4 - `inference.py`):
   ```python
   # Fine-tune memory settings:
   gpu_memory_utilization=0.95,  # Slightly reduce from 0.98
   max_model_len=2048,           # Reduce from 2836 if sequences are shorter
   ```

### **Advanced (If You Have Shorter Sequences):**
6. **Reduce Max Length** (Cell 4 - `inference.py`):
   ```python
   max_model_len=1536,  # Change from 2836 if your prompts are shorter
   ```

## 💡 **Expected Speed Improvements:**
- **Quick Wins**: 25-40% faster inference
- **All Changes**: 40-60% faster inference
- **Trade-off**: Minimal accuracy impact